Goal: gain intuition for different types of statistic we compute for a given peak

Parameters (everything here assumes a particular target)
- Scale factor $s$: FVP input / DMSO input
- Total reads $n_\mathrm{total, DMSO}$: over the entire genome, # DMSO reads
- Total reads $n_\mathrm{total, FVP}$: over the entire genome, # FVP reads
- Peak ratio $r$: over a given peak, # FVP reads / (# FVP reads + # DMSO reads)
- Total reads in peak $n_\mathrm{peak}$: over a given peak, # FVP reads + # DMSO reads

Statistics

| Method | $k$ | $n$ | $p$ |
| ------ | --- | --- | --- |
| MACS   | $r n_\mathrm{peak}$ | $n_\mathrm{total, FVP}$ | $\frac{(1 - r) n_\mathrm{peak}}{n_\mathrm{total, DMSO}}$ |
| Mitch1 | $r n_\mathrm{peak}$ | $n_\mathrm{peak}$ | $1 - r$ |
| Mitch2 | $r n_\mathrm{peak}$ | $n_\mathrm{peak}$ | $\frac{s}{1 + s}$ |

In [11]:
import numpy as np
import scipy
import pandas as pd
import plotly.express as px

In [17]:
n_total_dmso = int(3e6)
n_total_fvp = int(3e6)
rs = np.linspace(0.01, 0.99, 99) # 0.01, 0.02, ..., 0.99
n_peaks = np.linspace(10, 100, 10) # 10, 20, ..., 100
s = 0.7
pseudocount = 1

In [68]:
results = []
for pseudocount in [0, 1, 5]:
    for r in rs:
        for n_peak in n_peaks:
            k = int(round(r * n_peak)) + pseudocount

            # macs
            n_macs = n_total_fvp
            p_macs = ((1 - r) * n_peak + pseudocount) / n_total_dmso
            nlogp = max(-scipy.stats.binom.logcdf(k=k, n=n_macs, p=p_macs), -scipy.stats.binom.logsf(k=k, n=n_macs, p=p_macs))
            results.append(dict(r=r, n_peak=n_peak, nlogp=nlogp, method='macs', pseudocount=pseudocount))

            # mitch1
            n_mitch1 = n_peak
            p_mitch1 = 1 - r
            nlogp = max(-scipy.stats.binom.logcdf(k=k, n=n_mitch1, p=p_mitch1), -scipy.stats.binom.logsf(k=k, n=n_mitch1, p=p_mitch1))
            results.append(dict(r=r, n_peak=n_peak, nlogp=nlogp, method='mitch1', pseudocount=pseudocount))

            # mitch2
            n_mitch2 = n_peak
            p_mitch2 = s / (1 + s)
            nlogp = max(-scipy.stats.binom.logcdf(k=k, n=n_mitch2, p=p_mitch2), -scipy.stats.binom.logsf(k=k, n=n_mitch2, p=p_mitch2))
            results.append(dict(r=r, n_peak=n_peak, nlogp=nlogp, method='mitch2', pseudocount=pseudocount))

df = pd.DataFrame(results)

In [69]:
fig = px.scatter(
    (
        df
        .groupby(['method', 'pseudocount'], observed=True)[df.columns]
        .apply(lambda g: g.assign(nlogp=g['nlogp'].map(lambda x: x if np.isfinite(x) else g.loc[g['nlogp'].map(np.isfinite), 'nlogp'].max() + 1)))
        .pipe(lambda tb: tb.assign(**{'r_norm': tb['r'] / (s / (1 + s))}))
    ),
    x='r_norm',
    y='nlogp',
    color='n_peak',
    facet_row='method',
    facet_col='pseudocount',
    color_continuous_scale='cividis'
)
# fig.update_yaxes(matches=None)
fig.update_layout(
    height=1000,
    width=1000,
    yaxis_range=[0, 10]
)

In [67]:
fig = px.scatter(
    (
        df
        .groupby(['method', 'pseudocount'], observed=True)[df.columns]
        .apply(lambda g: g.assign(nlogp=g['nlogp'].map(lambda x: x if np.isfinite(x) else g.loc[g['nlogp'].map(np.isfinite), 'nlogp'].max() + 1)))
        .pipe(lambda tb: tb.assign(**{'r_norm': tb['r'] / (s / (1 + s))}))
    ),
    x='r_norm',
    y='nlogp',
    color='n_peak',
    facet_row='method',
    facet_col='pseudocount',
    color_continuous_scale='cividis'
)
# fig.update_yaxes(matches=None)
fig.update_layout(
    height=1000,
    width=1000,
    yaxis_range=[0, 10]
)

In [32]:
df.loc[~np.isfinite(df['nlogp'].values), 'method'].unique()

array(['mitch1', 'mitch2'], dtype=object)

In [34]:
df

,r,n_peak,nlogp,method
0,0.01,10.0,8.423478,macs
1,0.01,10.0,39.152987,mitch1
2,0.01,10.0,3.226841,mitch2
3,0.01,20.0,17.718156,macs
4,0.01,20.0,84.512047,mitch1
...,...,...,...,...
2965,0.99,90.0,inf,mitch1
2966,0.99,90.0,inf,mitch2
2967,0.99,100.0,300.328449,macs
2968,0.99,100.0,inf,mitch1


In [45]:
scipy.stats.binom.logcdf(k=20, n=20, p=p_mitch2)

0.0

In [33]:
k

100